## 파라미터 초기화
퍼셉트론에 필요한 파라미터 변수를 무작위로 초기화하자
- 입력값: X
- 가중치: W
- 편향치: b
- 목표값: y

In [ ]:
import numpy as np
np.random.seed(42)

X = np.array([0.5, 0.5, 0.5])  # 입력값 = [0.5, 0.5, 0.5]
W = np.random.rand(3)          # 가중치 3개 무작위 초기화
b = np.random.rand(1)          # 편향치 1개 무작위 초기화
y = np.array([-1])             # 목표값 = -1

In [ ]:
print("퍼셉트론 파라미터 초기값...")
print("W:", W)
print("b:", b)

## 예측 함수
입력값을 가중치와 곱하고 편향치와 더하여 출력값 z를 계산하고,
임계 함수에 통과시켜 예측값 y^을 도출하는 함수를 구현하자

In [ ]:
# 예측 함수
def predict(X, W, b):
    z = np.dot(X, W) + b
    return z

# 임계 함수
def threshold(z):
    if z > 0:
        return np.array([1])
    else:
        return np.array([-1])

# 순전파 함수
def forward(X, W, b):
    z = predict(X, W, b)
    y_hat = threshold(z)
    return y_hat

In [ ]:
z = predict(X, W, b)
y_hat = threshold(z)
print("출력값 z =", z)
print("예측값 y^ =", y_hat)
print("목표값 y =", y)

## 학습 구현
모델의 가중치와 편향을 조절하여 예측 성능을 최적화하는 학습을 구현해보자

In [ ]:
learning_rate = 0.1
num_epochs = 10

for epoch in range(num_epochs):
    y_hat = forward(X, W, b)
    error = y - y_hat
    W = W + learning_rate * error * X
    b = b + learning_rate * error
    print(f"[Epoch #{epoch + 1}] Error = {error}, W = {W}, b = {b}")

In [ ]:
z = predict(X, W, b)
y_hat = forward(X, W, b)
print("Trained weights:", W)
print("Trained bias:", b)
print("Trained z:", z)
print("Predict y^:", y_hat)